In [3]:
import os

In [4]:
%pwd

'd:\\ML_Projects\\Deep Learning\\ZooScan\\reasearch'

In [5]:
os.chdir("../")

In [6]:
%pwd

'd:\\ML_Projects\\Deep Learning\\ZooScan'

In [12]:
# Entity : Entity is nothing but it's a return type of function

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [13]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir= config.unzip_dir

        )

        return data_ingestion_config

In [15]:
## Data Ingestion
import os
import urllib.request as request
import zipfile
from cnnClassifier.logging import logger
from cnnClassifier.utils.common import get_size

In [16]:
class DataIngestion: 
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")

        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)


            


In [17]:
## Data Ingestion Pipeline
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingetsion = DataIngestion(config=data_ingestion_config)
    data_ingetsion.download_file()
    data_ingetsion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-12 17:20:19,093: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-12 17:20:19,097: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-12 17:20:19,101: INFO: common: Created directory at: artifacts]
[2025-04-12 17:20:19,105: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-04-12 17:20:24,651: INFO: 3544682667: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 8477248
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "4291ba2c5b107965347a5118c102f9ac3e3fff957543362bae258088ec4117cd"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9107:3532AE:963E2:1A3331:67FA537B
Accept-Ranges: bytes
Date: Sat, 12 Apr 2025 11:50:20 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4730-BOM
X-Cache: MIS